### Search near satellites overpasses

This code search for satellites overpasses acquired during a given temporal gap (delta t) over a given point P. 

The actual version supports Sentinel-2, Landsat-8, Landsat-9 and Harmonised Landsat (HLS) sensors.

User should specify:
- the latitude/longitude of the point of interrest in EPSG:4326
- the period of interrest startDate/endDate
- the couple of satellites to considered 
- the maximum temporal gap between the selected acquisition (delta_t_max)

The code outputs a FataFrame of all the satellites aquisitions (from 2 different sensors) within at short period of time (delta t).

##### Example:

* For the geo-location 37.42189 N, 122.08412 W, we can find 4 Sentinel-2 images acquired whithin 1 day maximum of a Landsat-8 image between March 1st and August 30th, 2020, 

<ins>Details (code output)<ins>

| Collection_1 | lon_1 | lat_1 | id_1 | date_1 | Collection_2 | lon_2 | lat_2 | id_2 | date_2 |
| -------- | ------- |-------- | ------- |-------- | ------- |-------- | ------- |-------- | ------- |
|0|COPERNICUS/S2_SR_HARMONIZED |-122.084191|37.421928|20200504T184921_20200504T185732_T10SEG_0|2020-05-04|LANDSAT/LC08/C02/T1_L2|-122.084191|37.421928|LC08_044034_20200505_0|2020-05-05|
|1| COPERNICUS/S2_SR_HARMONIZED|-122.084191|37.421928|20200623T184921_20200623T185629_T10SEG_0|2020-06-23|LANDSAT/LC08/C02/T1_L2|-122.084191|37.421928|LC08_044034_20200622_0|2020-06-22|
|2|COPERNICUS/S2_SR_HARMONIZED|-122.084191|37.421928|20200708T184919_20200708T185408_T10SEG_0|2020-07-08|LANDSAT/LC08/C02/T1_L2|-122.084191|37.421928|LC08_044034_20200708_0|2020-07-08|
|3|COPERNICUS/S2_SR_HARMONIZED|-122.084191|37.421928|20200723T184921_20200723T185433_T10SEG_0|2020-07-23|LANDSAT/LC08/C02/T1_L2|-122.084191|37.421928|LC08_044034_20200724_0|2020-07-24|

    
    
    
    ----------------------------------
    Najib Djamai
    March 28th, 2024

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
from datetime import datetime
import eoImage
import pandas as pd
import numpy as np
import toolsHLS, toolsL8, toolsL9, toolsS2

In [24]:
def search_overpass_dates (sensor,mapBounds,startDate,endDate,outputScaleSize=20,factor=1):
    tools=eval('tools'+sensor)
    print('CollectionName: '+tools.CollectionName())
    productCollection =  ee.ImageCollection(tools.CollectionName()) \
                          .filterBounds(mapBounds) \
                          .filterDate(startDate, endDate) \
                          .filterMetadata(tools.Cloudcover(),'less_than',maxCloudcover) \
                          .limit(5000) \
                          .map(lambda image: image.clip(mapBounds)) \
                          .map(lambda image: tools.MaskClear(image))  \
                          .map(lambda image: eoImage.attach_Date(image)) \
                          .map(lambda image: eoImage.attach_LonLat(image)) 
                          
    productCollection = ee.ImageCollection(productCollection)
    outputScaleSize= ee.Number(outputScaleSize)
    sampleData = productCollection.map(lambda image: image.sample(region=mapBounds, projection=image.select('date').projection(), scale=outputScaleSize,geometries=True, dropNulls = True, factor=factor) ).flatten()
    sampleList=sampleData.getInfo()['features']
    
    sampleDF = pd.DataFrame(columns=['Collection','lon','lat','id','date'])
    for col in sampleList:
        sampleDF.loc[len(sampleDF)] = [tools.CollectionName(),col['geometry']['coordinates'][0],col['geometry']['coordinates'][1],col['id'],col['properties']['date']]
    sampleDF['date'] = pd.to_datetime(sampleDF['date'],unit='ms').dt.date 
    return (sampleDF)    

### 1 - Specifty point / area od interrest

##### a. Define parameters

In [29]:
latitude=37.42189
longitude=-122.08412
startDate='2020-05-01'
endDate='2020-08-30'
sensor1='S2'
sensor2='L8'
delta_t_max=1 # maximum accepted period between sensors overpasses [in days]
maxCloudcover=90

##### b. Processing

In [30]:
startDate = datetime.strptime(startDate,"%Y-%m-%d")
endDate =  datetime.strptime(endDate,"%Y-%m-%d")
mapBounds = ee.Geometry.Point([longitude, latitude])  # defaukt EPSG:4326

df1=search_overpass_dates (sensor1,mapBounds,startDate,endDate)
df2=search_overpass_dates (sensor2,mapBounds,startDate,endDate)

CollectionName: COPERNICUS/S2_SR_HARMONIZED
CollectionName: LANDSAT/LC08/C02/T1_L2


In [31]:
sampleDF = pd.DataFrame()
for index, row in df1.iterrows():
    idx=np.argmin(np.abs(df2['date']-row['date']))
    if (np.abs(df2['date'][idx]-row['date'])).days<=delta_t_max: 
        aux=pd.concat([row.add_suffix('_1'), df2.loc[idx,:].add_suffix('_2')], axis=0)
        aux=np.transpose(aux.to_frame())
        sampleDF=pd.concat([sampleDF,aux])
sampleDF=sampleDF.reset_index(drop=True)

##### c. Exporting

In [32]:
sampleDF

,Collection_1,lon_1,lat_1,id_1,date_1,Collection_2,lon_2,lat_2,id_2,date_2
0,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,20200504T184921_20200504T185732_T10SEG_0,2020-05-04,LANDSAT/LC08/C02/T1_L2,-122.084191,37.421928,LC08_044034_20200505_0,2020-05-05
1,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,20200623T184921_20200623T185629_T10SEG_0,2020-06-23,LANDSAT/LC08/C02/T1_L2,-122.084191,37.421928,LC08_044034_20200622_0,2020-06-22
2,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,20200708T184919_20200708T185408_T10SEG_0,2020-07-08,LANDSAT/LC08/C02/T1_L2,-122.084191,37.421928,LC08_044034_20200708_0,2020-07-08
3,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,20200723T184921_20200723T185433_T10SEG_0,2020-07-23,LANDSAT/LC08/C02/T1_L2,-122.084191,37.421928,LC08_044034_20200724_0,2020-07-24
